In [149]:
import re
import requests
import json
import colorsys
import csv
import math

from colr import color

reds = {} # reds, pinks, oranges
pinks = {} # reds, pinks, oranges
oranges = {} # reds, pinks, oranges
yellows = {} # just yellows
greens = {} # just greens
blues = {} # just blues
purples = {} # just purples
neutrals = {} # white, black, beige, browns

# define a basic set of colors based on human perception
# THIS IS SAD DON'T LOOK AT THIS
# Cats can have little a brute force
basic = {
            'colors': [
                        {'name': 'neutral', 'r' : 255, 'g' : 255, 'b' : 255}, #white
                        {'name': 'neutral', 'r' : 128, 'g' : 128, 'b' : 128}, #grey
                        {'name': 'neutral', 'r' : 192, 'g' : 192, 'b' : 192}, #silver
                        {'name': 'neutral', 'r' : 0, 'g' : 0, 'b' : 0}, #black
                
                        {'name': 'red', 'r' : 255, 'g' : 0, 'b' : 0},
                        {'name': 'red', 'r' : 128, 'g' : 0, 'b' : 0}, #maroon
                
                        {'name': 'yellow', 'r' : 255, 'g' : 255, 'b' : 0},
                
                        {'name': 'green', 'r' : 128, 'g' : 128, 'b' : 0}, #olive
                        {'name': 'green', 'r' : 0, 'g' : 255, 'b' : 0}, #lime
                        {'name': 'green', 'r' : 0, 'g' : 128, 'b' : 0},
                        {'name': 'green', 'r' : 0, 'g' : 128, 'b' : 128}, #teal
                
                        {'name': 'blue', 'r' : 0, 'g' : 255, 'b' : 255}, #aqua
                        {'name': 'blue', 'r' : 0, 'g' : 0, 'b' : 255 },
                        {'name': 'blue', 'r' : 0, 'g' : 0, 'b' : 128 },
                
                        {'name': 'pink', 'r' : 255, 'g' : 0, 'b' : 255 }, #fuschia
                
                        {'name': 'purple', 'r' : 128, 'g' : 0, 'b' : 128 },
                
                        ]
        }

def rgbAPIRequest(color1, color2, color3):
    """ Make calls to color API to verify color exists.
    """
    requestURL = "http://thecolorapi.com/id?rgb=" + str(color1) + "," + str(color2) + "," + str(color3)
    response = requests.get(requestURL)

    if response.status_code != 200:
        return None
    else:
        return response.json()
    
def hexAPIRequest(hexcode):
    """ Make calls to color API to verify color exists.
    """
    requestURL = "http://thecolorapi.com/id?hex=" + hexcode.replace("#", "")
    response = requests.get(requestURL)

    if response.status_code != 200:
        return None
    else:
        return response.json()
    
def testAPI():
    """ Testing the API.
    """
    # test = rgbAPIRequest(255, 242, 219)
    test = hexAPIRequest('#a59344')
    print(test)
    print(test['name']['exact_match_name']) # print if color match
    print(test['name']['value']) # print color name
    print(test['hex']['value']) #hex code value
    print(test['rgb']['value'])
    print(test['rgb']['r'])
    print(test['rgb']['g'])
    print(test['rgb']['b'])
    
def readTXT():
    """ Reading a text file. Might not even be used because no white or beige...
    """
    for line in open("satfaces.txt"):
        splitted = re.split('] ', line)
        nums = splitted[0].replace("[", "")
        sep_nums = nums.split(", ")
        color = splitted[1].strip()
        break
        

def thisWasABadIdea():
    """ Not the smartest moment I had
    """
    counter = 0
    with open('colornames.csv', encoding="utf8") as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            
            if counter > 0:
                # Just realizing that for 18,000 rows, this will take forever....
                res = hexAPIRequest(row[1])
                
                if res != None and res['name']['exact_match_name'] == True:
                    s = res['hsv']['s']
                    h = res['hsv']['h']
                    v = res['hsv']['v']
                    
                    colorname = res['name']['value']
                    hexcode = res['hex']['value']
                    rgb = res['rgb']['value']
                    
                    if (17 < h < 60 and s < 15) or (s < 3) or (v < 15) or (s < 10 and v <35):
                        addToDict(neutrals, colorname, rgb, hexcode)
                    elif (0 < h < 41) or (296 < h < 359):
                        addToDict(reds, colorname, rgb, hexcode)
                    elif (42 < h < 61):
                        addToDict(yellows, colorname, rgb, hexcode)
                    elif (62 < h < 172):
                        addToDict(greens, colorname, rgb, hexcode)
                    elif (173 < h < 254):
                        addToDict(blues, colorname, rgb, hexcode)
                    elif (255 < h < 295):
                        addToDict(purples, colorname, rgb, hexcode)
                else:
                    continue
            counter = counter + 1

# ---------------- Functions above this line were ideas that did not work -------------- #
            
def addToDict(dictionary, colorName, rgb, hexcode):
    """ Adds color to specified dictionary.
    """
    dictionary[colorName] = {
                             'rgb' : rgb,
                             'hex' : hexcode
                            }

def findClosestColor(color_obj):
    """ Using a clever formula from stack overflow
        Human adjusted formula for roygbiv
    """
    nearestColor = ""
    nearestDist = float('inf')
    for colors in basic['colors']:
        r_mean = (float(color_obj['r']) + float(colors['r'])) / 2
        
        r = (float(color_obj['r']) - float(colors['r']))
        g = (float(color_obj['g']) - float(colors['g']))
        b = (float(color_obj['b']) - float(colors['b']))
        dist = math.sqrt((((512 + r_mean) * r * r) / 256) + 4 * g * g + (((767 - r_mean) * b * b) / 256))
        
        if dist < nearestDist:
            nearestDist = dist
            nearestColor = colors['name']
            
    return nearestColor
    
    
# alright fuck the API request let's do this manually
def partitionJSON():
    """ Breaking up colors into correct dictionaries
    """
    with open('colorNames.json', encoding='utf-8') as fh:
        colors_dict = json.load(fh)
    print(len(colors_dict['colors']))
    
    for color in colors_dict['colors']:
        colorName = color['name']
        colorRGB = [color['r'], color['g'], color['b']]
        colorHex = "#" + color['hex']
        closest_color = findClosestColor(color)
        
        if closest_color == "red":
            addToDict(reds, colorName, colorRGB, colorHex)
        elif closest_color == "orange":
            addToDict(oranges, colorName, colorRGB, colorHex)
        elif closest_color == "pink":
            addToDict(pinks, colorName, colorRGB, colorHex)
        elif closest_color == "yellow":
            addToDict(yellows, colorName, colorRGB, colorHex)
        elif closest_color == "green":
            addToDict(greens, colorName, colorRGB, colorHex)
        elif closest_color == "blue":
            addToDict(blues, colorName, colorRGB, colorHex)
        elif closest_color == "purple":
            addToDict(purples, colorName, colorRGB, colorHex)
        else:
            addToDict(neutrals, colorName, colorRGB, colorHex)
            
    with open('attempt2/red.json', 'w', encoding='utf-8') as f:
        json.dump(reds, f, ensure_ascii=False, indent=4)
    
    with open('attempt2/orange.json', 'w', encoding='utf-8') as f:
        json.dump(oranges, f, ensure_ascii=False, indent=4)
        
    with open('attempt2/pink.json', 'w', encoding='utf-8') as f:
        json.dump(pinks, f, ensure_ascii=False, indent=4)
    
    with open('attempt2/yellow.json', 'w', encoding='utf-8') as f:
        json.dump(yellows, f, ensure_ascii=False, indent=4)
    
    with open('attempt2/blue.json', 'w', encoding='utf-8') as f:
        json.dump(blues, f, ensure_ascii=False, indent=4)
    
    with open('attempt2/green.json', 'w', encoding='utf-8') as f:
        json.dump(greens, f, ensure_ascii=False, indent=4)
        
    with open('attempt2/purple.json', 'w', encoding='utf-8') as f:
        json.dump(purples, f, ensure_ascii=False, indent=4)
        
    with open('attempt2/neutral.json', 'w', encoding='utf-8') as f:
        json.dump(neutrals, f, ensure_ascii=False, indent=4)

        
def checkForDuplicates(path1, path2):
    """ Check for mistakes I made :(
    """
    with open(path1, encoding='utf-8') as fh:
        f1 = json.load(fh)
        
    with open(path2, encoding='utf-8') as fh:
        f2 = json.load(fh)
    
    nameset = set()
    for c in f1:
        nameset.add(c)
        
    for c in f2:
        if c in nameset:
            print(c, f2[c]['hex'])
    print(color("-- Done running. --", fore=(255, 0, 0), back='#fff'))

        
def printColorFromJSON(json_filepath):
    """ Check results of partitioned JSONs.
    """
    with open(json_filepath, encoding='utf-8') as fh:
        colors = json.load(fh)
        
    print("Num entries is ", len(colors))
        
    for c in colors:
        print_str = ' Hello world.  \n         ' + c + ' \n       ' + colors[c]['hex'] + ' \n               '
        r = colors[c]['rgb'][0]
        g = colors[c]['rgb'][1]
        b = colors[c]['rgb'][2]
        print(color(print_str, fore='fff', back=(r, g, b)))
        print()
    

# Change depending on which json you want to look at.
printColorFromJSON('color_dicts/neutral.json')

1839
Num entries is  576
 Hello world.  
         Black 
       #000000 
               

 Hello world.  
         Midnight Moss 
       #041004 
               

 Hello world.  
         Pink Swan 
       #BEB5B7 
               

 Hello world.  
         Jaguar 
       #080110 
               

 Hello world.  
         Black Russian 
       #0A001C 
               

 Hello world.  
         Cod Gray 
       #0B0B0B 
               

 Hello world.  
         Marshland 
       #0B0F08 
               

 Hello world.  
         Black Forest 
       #0B1304 
               

 Hello world.  
         Ebony 
       #0C0B1D 
               

 Hello world.  
         Woodsmoke 
       #0C0D0F 
               

 Hello world.  
         Bunker 
       #0D1117 
               

 Hello world.  
         Cinder 
       #0E0E18 
               

 Hello world.  
         Diesel 
       #130000 
               

 Hello world.  
         Asphalt 
       #130A06 
               

 Hello world.  
     

 Hello world.  
         Quick Silver 
       #A6A6A6 
               

 Hello world.  
         Onyx 
       #353839 
               

 Hello world.  
         Black Shadows 
       #BFAFB2 
               

 Hello world.  
         Ash Grey 
       #CDC5C2 
               

 Hello world.  
         Silver 
       #C0C0C0 
               

 Hello world.  
         Smokey Topaz 
       #832A0D 
               

 Hello world.  
         Sonic Silver 
       #757575 
               

 Hello world.  
         Salt Box 
       #685E6E 
               

 Hello world.  
         Sweet Brown 
       #A83731 
               

 Hello world.  
         Tigers Eye 
       #B56917 
               

